In [1]:
import torch
from PIL import Image
import open_clip

c:\Users\bezgr\STUDY\HSE\Course 2\Coursework\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
MODEL_NAME = 'ViT-B-32'
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', load_weights_only=False)
tokenizer = open_clip.get_tokenizer(MODEL_NAME)

UnpicklingError: invalid load key, '\x08'.

In [ ]:
def match_image_text(image_path: str, text: str) -> float:
    """
    Оценивает вероятность соответствия текстового описания изображению.
    
    :param image_path: Путь к изображению.
    :param text: Текстовое описание на русском языке.
    :return: Вероятность соответствия в процентах.
    """
    # Загружаем и обрабатываем изображение
    image = preprocess(Image.open(image_path)).unsqueeze(0)
    
    # Токенизируем текст
    text_tokens = tokenizer([text])
    
    # Вычисляем эмбеддинги
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_tokens)
    
    # Нормализация эмбеддингов
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    
    # Считаем косинусное сходство
    similarity = (image_features @ text_features.T).item()

    # Преобразуем в вероятность (0-100%)
    probability = (similarity + 1) / 2 * 100  # Масштабируем в диапазон [0, 100]

    return probability

In [3]:
image_path = "App/media/preview/preview_62c642b6-122c-4a3b-b628-2ae61542be1d.jpg"  # Замените на путь к изображению
text = "Красивая белая кошка на фоне природы"
probability = match_image_text(image_path, text)
print(f"Вероятность соответствия: {probability:.2f}%")

Downloading: 100%|██████████| 605M/605M [01:50<00:00, 5.48MB/s] 


UnpicklingError: Weights only load failed. In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Unsupported operand 232

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.